In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt

In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
urls = ["http://www.focus-news.net/RSS/channel_0.xml",
    ]
feeds = [feedparser.parse(url)['entries'] for url in urls]
for url in urls:
    print(url)
feed = [item for feed in feeds for item in feed]

In [ ]:
for newslink in feed:
  try:
    nlink=newslink["link"]
    collection_name = dbname["focus_scrape_1"]
    if list(collection_name.find({"news_link" : nlink })):
        print("Article Exists")
    else:
        dt_obj=newslink["published"]
        date_posted = dt.strptime(dt_obj, '%a, %d %b %Y %H:%M:%S %z').timestamp()
        news_title=newslink["title"]
        text=newslink["summary"]
        page = requests.get(nlink)
        data = page.text
        soup = BeautifulSoup(data, 'html.parser')
        text = soup.find('div', class_='news-text').get_text()
        text = text.replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
        news_payload = {
        "date_posted" : date_posted,
        "news_link" : nlink,
        "news_text" : text,
        "news_title" : news_title
        }
        collection_name.insert_many([news_payload])
        print(news_payload)
  except:
    pass